In [1]:
import torch
import torch.nn as nn
from tensorboardX import SummaryWriter
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(     #input_size=(1*28*28)
            nn.Conv2d(1, 6, 5, 1, 2),
            nn.ReLU(),      #(6*28*28)
            nn.MaxPool2d(kernel_size=2, stride=2),  #output_size=(6*14*14)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),      #(16*10*10)
            nn.MaxPool2d(2, 2)  #output_size=(16*5*5)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(84, 10)

    # 定义前向传播过程，输入为x
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # nn.Linear()的输入输出都是维度为一的值，所以要把多维度的tensor展平成一维
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

dummy_input = torch.rand(13, 1, 28, 28) #假设输入13张1*28*28的图片
model = LeNet()
with SummaryWriter(comment='LeNet') as w:
    w.add_graph(model, (dummy_input, ))

In [1]:
import torch
from torch.autograd import Variable

from graphviz import Digraph

def make_dot(var, params=None):
    """
    画出 PyTorch 自动梯度图 autograd graph 的 Graphviz 表示.
    蓝色节点表示有梯度计算的变量Variables;
    橙色节点表示用于 torch.autograd.Function 中的 backward 的张量 Tensors.

    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    if params is not None:
        assert all(isinstance(p, Variable) for p in params.values())
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style='filled', shape='box', align='left',
                              fontsize='12', ranksep='0.1', height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return '(' + (', ').join(['%d' % v for v in size]) + ')'

    output_nodes = (var.grad_fn,) if not isinstance(var, tuple) else tuple(v.grad_fn for v in var)

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                # note: this used to show .saved_tensors in pytorch0.2, but stopped
                # working as it was moved to ATen and Variable-Tensor merged
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            elif var in output_nodes:
                dot.node(str(id(var)), str(type(var).__name__), fillcolor='darkolivegreen1')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)

    # 多输出场景 multiple outputs
    if isinstance(var, tuple):
        for v in var:
            add_nodes(v.grad_fn)
    else:
        add_nodes(var.grad_fn)
    return dot

In [2]:
from torch import tensor
import torch
import math
import numpy as np
from torch import nn
from ResNet import ResNet
from tensorboardX import SummaryWriter

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(     #input_size=(1*28*28)
            nn.Conv2d(1, 6, 5, 1, 2),
            nn.ReLU(),      #(6*28*28)
            nn.MaxPool2d(kernel_size=2, stride=2),  #output_size=(6*14*14)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),      #(16*10*10)
            nn.MaxPool2d(2, 2)  #output_size=(16*5*5)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(84, 10)
 
    # 定义前向传播过程，输入为x
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # nn.Linear()的输入输出都是维度为一的值，所以要把多维度的tensor展平成一维
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
 
dummy_input = torch.rand(13, 1, 28, 28) #假设输入13张1*28*28的图片
model = LeNet()
with SummaryWriter(comment='LeNet') as w:
    w.add_graph(model, (dummy_input, ))

In [9]:
net = ResNet(3).double()
x = tensor(np.zeros((128, 3, 32, 32))).double()
# with SummaryWriter(comment='ResNet') as w:
#     w.add_graph(net, (x, ))
vise_graph = make_dot(net(x), params=dict(net.named_parameters()))
vise_graph.render('test-output/test-table.gv')
# y = net(x)
# y.shape

'test-output/test-table.gv.pdf'

In [2]:
class DropoutFMP(nn.Module):
    "fractional max pooling with dropout"
    def __init__(self, size, out_channels, dropout=0):
        super(DropoutFMP, self).__init__()
        self.norm = LayerNorm(features=size)
        self.dropout = nn.Dropout(dropout)
        self.fmp = FMPBlock(size[0], out_channels)
    
    def forward(self, x):
        x = self.norm(x)
        x = self.fmp(x)
        x = self.dropout(x)
        return x

In [25]:
class FMPNet(nn.Module):
    "implemnet of a cnn network with fractional max pooling"
    def __init__(self):
        super(FMPNet, self).__init__()
        n = 1
        m = 2
        self.input = DropoutFMP((3, 32, 32), n)
        layers = []
        h = 25  # height
        k = 1   # times of 160 channels
        while h >= 2:
            ne = DropoutFMP((n*k, h, h), n*(k+1), 0.045*k)
            k += 1
            h = int(0.8 * h)
            layers.append(ne)
        self.layers = nn.Sequential(*layers)
        self.l1 = nn.Linear(n*11, m)
        self.l2 = nn.Linear(m, 10)
    
    def forward(self, x):
        x = self.input(x)
        x = self.layers(x)
        b = x.size()[0]
        return self.l2(self.l1(x.view(b, -1)))

In [26]:
net = FMPNet().double().eval()
# x = tensor(np.zeros((1, 3, 32, 32))).double()
# y = net(x)
# y.shape

In [27]:
print(net)

FMPNet(
  (input): DropoutFMP(
    (norm): LayerNorm()
    (dropout): Dropout(p=0)
    (fmp): FMPBlock(
      (c1): Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (fmp): FractionalMaxPool2d()
    )
  )
  (layers): Sequential(
    (0): DropoutFMP(
      (norm): LayerNorm()
      (dropout): Dropout(p=0.045)
      (fmp): FMPBlock(
        (c1): Conv2d(1, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (fmp): FractionalMaxPool2d()
      )
    )
    (1): DropoutFMP(
      (norm): LayerNorm()
      (dropout): Dropout(p=0.09)
      (fmp): FMPBlock(
        (c1): Conv2d(2, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (fmp): FractionalMaxPool2d()
      )
    )
    (2): DropoutFMP(
      (norm): LayerNorm()
      (dropout): Dropout(p=0.135)
      (fmp): FMPBlock(
        (c1): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (fmp): FractionalMaxPool2d()
      )
    )
    (3): DropoutFMP(
      (norm): LayerNorm()
      

In [28]:
x = tensor(np.zeros((1, 3, 32, 32))).double()
y = net(x)
y.shape

torch.Size([1, 10])

In [10]:
h = 32  # height
channel = 0
while h >=1:
#     ne = DropoutFMP((channel, h, h), channel + 160)
    channel += 1
    print(h)
    h = math.floor(0.8 * h)
channel

32
25
20
16
12
9
7
5
4
3
2
1


12

In [23]:
i += 1
y = net(y)
y.shape

torch.Size([1, 128, 3, 3])

In [14]:
i

10